In [16]:
import math
import random
import time
from itertools import chain
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from scipy.sparse.csgraph import shortest_path
from sklearn.metrics import roc_auc_score, average_precision_score
from torch.nn import BCEWithLogitsLoss, Conv1d, MaxPool1d, ModuleList
from torch_geometric.data import Data, InMemoryDataset
from torch_geometric.datasets import Planetoid
from torch_geometric.loader import DataLoader
from torch_geometric.nn import MLP, GCNConv, SortAggregation
from torch_geometric.transforms import RandomLinkSplit
from torch_geometric.utils import k_hop_subgraph, to_scipy_sparse_matrix

In [ ]:
# class SEALDataset(InMemoryDataset):
#     def __init__(self, dataset, num_hops, top_n, scorer, split='train'):
#         self._data = dataset[0]
#         self.num_hops = num_hops
#         self.top_n = top_n
#         self.scorer = scorer
#         super().__init__(dataset.root)
#         index = ['train', 'val', 'test'].index(split)
#         self.load(self.processed_paths[index])

#     @property
#     def processed_file_names(self):
#         return ['SEAL++_train_data.pt', 'SEAL++_val_data.pt', 'SEAL++_test_data.pt']

#     def process(self):
#         transform = RandomLinkSplit(num_val=0.05, num_test=0.1,
#                                     is_undirected=True, split_labels=True)
#         train_data, val_data, test_data = transform(self._data)

#         self._max_z = 0

#         # Collect a list of subgraphs for training, validation and testing:
#         train_pos_data_list = self.extract_enclosing_subgraphs(
#             train_data.edge_index, train_data.pos_edge_label_index, 1)
#         train_neg_data_list = self.extract_enclosing_subgraphs(
#             train_data.edge_index, train_data.neg_edge_label_index, 0)

#         val_pos_data_list = self.extract_enclosing_subgraphs(
#             val_data.edge_index, val_data.pos_edge_label_index, 1)
#         val_neg_data_list = self.extract_enclosing_subgraphs(
#             val_data.edge_index, val_data.neg_edge_label_index, 0)

#         test_pos_data_list = self.extract_enclosing_subgraphs(
#             test_data.edge_index, test_data.pos_edge_label_index, 1)
#         test_neg_data_list = self.extract_enclosing_subgraphs(
#             test_data.edge_index, test_data.neg_edge_label_index, 0)

#         # Convert node labeling to one-hot features.
#         for data in chain(train_pos_data_list, train_neg_data_list,
#                           val_pos_data_list, val_neg_data_list,
#                           test_pos_data_list, test_neg_data_list):
#             # We solely learn links from structure, dropping any node features:
#             data.x = F.one_hot(data.z, self._max_z + 1).to(torch.float)

#         train_data_list = train_pos_data_list + train_neg_data_list
#         self.save(train_data_list, self.processed_paths[0])
#         val_data_list = val_pos_data_list + val_neg_data_list
#         self.save(val_data_list, self.processed_paths[1])
#         test_data_list = test_pos_data_list + test_neg_data_list
#         self.save(test_data_list, self.processed_paths[2])

#     def prune_subgraph(self, src, dst, data):
#         sub_nodes, sub_edge_index, mapping, _ = k_hop_subgraph(
#             [src, dst], self.num_hops, data.edge_index, relabel_nodes=True)
#         x_sub = data.x[sub_nodes]

#         #打分
#         with torch.no_grad():
#             score = self.scorer(x_sub)

#         #选top n
#         top_n = min(self.top_n, x_sub.size(0))
#         idx_top = score.topk(top_n).indices

#         mask = torch.zeros_like(score, dtype=torch.bool)
#         mask[idx_top] = True
#         sub_nodes_new = sub_nodes[mask]

#         keep = mask[sub_edge_index[0]] & mask[sub_edge_index[1]]
#         sub_edge_index_new = sub_edge_index[:, keep]

#         return sub_nodes_new, sub_edge_index_new, mapping

#     def extract_enclosing_subgraphs(self, data, edge_label_index, y):
#         data_list = []
#         for src, dst in edge_label_index.t().tolist():

#             sub_nodes_new, sub_edge_index_new, mapping = self.prune_subgraph(self, src, dst, data)
#             src, dst = mapping.tolist()

#             # Remove target link from the subgraph.
#             mask1 = (sub_edge_index_new[0] != src) | (sub_edge_index_new[1] != dst)
#             mask2 = (sub_edge_index_new[0] != dst) | (sub_edge_index_new[1] != src)
#             sub_edge_index_new = sub_edge_index_new[:, mask1 & mask2]

#             # Calculate node labeling.
#             z = self.drnl_node_labeling(sub_edge_index_new, src, dst,
#                                         num_nodes=sub_nodes_new.size(0))

#             data = Data(x=self._data.x[sub_nodes_new], z=z,
#                         edge_index=sub_edge_index_new, y=y)
#             data_list.append(data)

#         return data_list

#     def drnl_node_labeling(self, edge_index, src, dst, num_nodes=None):
#         # Double-radius node labeling (DRNL).
#         src, dst = (dst, src) if src > dst else (src, dst)
#         adj = to_scipy_sparse_matrix(edge_index, num_nodes=num_nodes).tocsr()

#         idx = list(range(src)) + list(range(src + 1, adj.shape[0]))
#         adj_wo_src = adj[idx, :][:, idx]

#         idx = list(range(dst)) + list(range(dst + 1, adj.shape[0]))
#         adj_wo_dst = adj[idx, :][:, idx]

#         dist2src = shortest_path(adj_wo_dst, directed=False, unweighted=True,
#                                  indices=src)
#         dist2src = np.insert(dist2src, dst, 0, axis=0)
#         dist2src = torch.from_numpy(dist2src)

#         dist2dst = shortest_path(adj_wo_src, directed=False, unweighted=True,
#                                  indices=dst - 1)
#         dist2dst = np.insert(dist2dst, src, 0, axis=0)
#         dist2dst = torch.from_numpy(dist2dst)

#         dist = dist2src + dist2dst
#         dist_over_2, dist_mod_2 = dist // 2, dist % 2

#         z = 1 + torch.min(dist2src, dist2dst)
#         z += dist_over_2 * (dist_over_2 + dist_mod_2 - 1)
#         z[src] = 1.
#         z[dst] = 1.
#         z[torch.isnan(z)] = 0.

#         self._max_z = max(int(z.max()), self._max_z)

#         return z.to(torch.long)

In [15]:
def drnl_node_labeling(edge_index, src, dst, num_nodes=None):
    # Double-radius node labeling (DRNL).
    src, dst = (dst, src) if src > dst else (src, dst)
    adj = to_scipy_sparse_matrix(edge_index, num_nodes=num_nodes).tocsr()

    idx = list(range(src)) + list(range(src + 1, adj.shape[0]))
    adj_wo_src = adj[idx, :][:, idx]

    idx = list(range(dst)) + list(range(dst + 1, adj.shape[0]))
    adj_wo_dst = adj[idx, :][:, idx]

    dist2src = shortest_path(adj_wo_dst, directed=False, unweighted=True,
                                indices=src)
    dist2src = np.insert(dist2src, dst, 0, axis=0)
    dist2src = torch.from_numpy(dist2src)

    dist2dst = shortest_path(adj_wo_src, directed=False, unweighted=True,
                                indices=dst - 1)
    dist2dst = np.insert(dist2dst, src, 0, axis=0)
    dist2dst = torch.from_numpy(dist2dst)

    dist = dist2src + dist2dst
    dist_over_2, dist_mod_2 = dist // 2, dist % 2

    z = 1 + torch.min(dist2src, dist2dst)
    z += dist_over_2 * (dist_over_2 + dist_mod_2 - 1)
    z[src] = 1.
    z[dst] = 1.
    z[torch.isnan(z)] = 0.

    _max_z = max(int(z.max()), _max_z)

    return z.to(torch.long)

In [ ]:
def extract_enclosing_subgraphs(batch_edges, data, scorer, num_hops, top_n):
    """
    针对一批目标边动态裁剪子图并打包成DataList
    :param batch_edges: [batch_size, 2] 目标边 (src, dst) 对
    :param data: 全图 PyG Data
    :param scorer: MLP/attention网络
    :param num_hops: k-hop
    :param top_n: Top-N邻居数量
    :return: DataList
    """
    data_list = []
    for (src, dst, label) in batch_edges:  # label: 1/0
        # 采k-hop子图
        node_idx, edge_index, mapping, _ = k_hop_subgraph(
            [src, dst], num_hops, data.edge_index, relabel_nodes=True, num_nodes=data.num_nodes
        )
        x_sub = data.x[node_idx]  # [num_nodes_sub, feat_dim]
        # scorer打分（训练时要参与反向传播，所以不能no_grad！）
        score = scorer(x_sub)
        # Top-N
        n = min(top_n, x_sub.size(0))
        idx_top = score.topk(n).indices
        mask = torch.zeros_like(score, dtype=torch.bool)
        mask[idx_top] = True
        node_idx_new = node_idx[mask]
        # edge筛选
        keep = mask[edge_index[0]] & mask[edge_index[1]]
        edge_index_new = edge_index[:, keep]
        # relabel（节点index要重新映射成0~N-1）
        node_map = {old.item(): i for i, old in enumerate(node_idx_new)}
        edge_index_new = torch.tensor(
            [[node_map[n.item()] for n in edge_index_new[0]],
             [node_map[n.item()] for n in edge_index_new[1]]],
            dtype=torch.long
        )
        # DRNL标号
        z = drnl_node_labeling(edge_index_new, mapping[0], mapping[1], num_nodes=node_idx_new.size(0))
        # 构造Data对象
        data_sub = Data(
            x=x_sub[mask],
            z=z,
            edge_index=edge_index_new,
            y=torch.tensor([label], dtype=torch.float),
        )
        data_list.append(data_sub)
    return data_list

In [8]:
class NeighborScorer(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super().__init__()
        self.mlp = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, 1)
        )

    def forward(self, x):
        
        score = self.mlp(x).squeeze(-1)
        return score

In [ ]:
# scorer = NeighborScorer(input_dim=128, hidden_dim=64)
# dataset = Planetoid('./data/Planetoid', name='Cora')

# train_dataset = SEALDataset(dataset, num_hops=2, top_n=10, scorer=scorer, split='train')
# val_dataset = SEALDataset(dataset, num_hops=2, top_n=10, scorer=scorer, split='val')
# test_dataset = SEALDataset(dataset, num_hops=2, top_n=10, scorer=scorer, split='test')

# train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
# val_loader = DataLoader(val_dataset, batch_size=32)
# test_loader = DataLoader(test_dataset, batch_size=32)

In [19]:
dataset = Planetoid('./data/Planetoid', name='Cora')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
data = dataset[0].to(device)

# 你可以用RandomLinkSplit, train_test_split_edges等采样正负边
# 假设已生成 train_edges_list: [(src, dst, label), ...]，label=1/0
# 这里只演示如何生成全部正/负例边列表
from torch_geometric.utils import negative_sampling

# 正例
edge_index = data.edge_index
num_pos = edge_index.size(1)
pos_edge_list = [(edge_index[0, i].item(), edge_index[1, i].item(), 1) for i in range(num_pos)]

# 负例
neg_edge_index = negative_sampling(
    edge_index=edge_index, num_nodes=data.num_nodes, num_neg_samples=num_pos
)
neg_edge_list = [(neg_edge_index[0, i].item(), neg_edge_index[1, i].item(), 0) for i in range(num_pos)]

# 合并打乱
all_edges = pos_edge_list + neg_edge_list
random.shuffle(all_edges)


In [20]:
class DGCNN(nn.Module):
    def __init__(self, hidden_dim, num_layers, dataset, GNN=GCNConv, k=0.6):
        super().__init__()

        if k < 1:  # Transform percentile to number.
            num_nodes = sorted([data.num_nodes for data in dataset])
            k = num_nodes[int(math.ceil(k * len(num_nodes))) - 1]
            k = int(max(10, k))

        self.convs = ModuleList()
        self.convs.append(GNN(dataset.num_features, hidden_dim))
        for i in range(0, num_layers - 1):
            self.convs.append(GNN(hidden_dim, hidden_dim))
        self.convs.append(GNN(hidden_dim, 1))

        conv1d_channels = [16, 32]
        total_latent_dim = hidden_dim * num_layers + 1
        conv1d_kws = [total_latent_dim, 5]
        self.conv1 = Conv1d(1, conv1d_channels[0], conv1d_kws[0],
                            conv1d_kws[0])
        self.pool = SortAggregation(k)
        self.maxpool1d = MaxPool1d(2, 2)
        self.conv2 = Conv1d(conv1d_channels[0], conv1d_channels[1],
                            conv1d_kws[1], 1)
        dense_dim = int((k - 2) / 2 + 1)
        dense_dim = (dense_dim - conv1d_kws[1] + 1) * conv1d_channels[1]
        self.mlp = MLP([dense_dim, 128, 1], dropout=0.5, norm=None)

    def forward(self, x, edge_index, batch):
        xs = [x]
        for conv in self.convs:
            xs += [conv(xs[-1], edge_index).tanh()]
        x = torch.cat(xs[1:], dim=-1)

        # Global pooling.
        x = self.pool(x, batch)
        x = x.unsqueeze(1)  # [num_graphs, 1, k * hidden]
        x = self.conv1(x).relu()
        x = self.maxpool1d(x)
        x = self.conv2(x).relu()
        x = x.view(x.size(0), -1)  # [num_graphs, dense_dim]

        return self.mlp(x)


In [ ]:
# class DGCNN(nn.Module):
#     def __init__(self, hidden_dim, num_layers, GNN=GCNConv, k=0.6):
#         super().__init__()

#         if k < 1:  # Transform percentile to number.
#             num_nodes = sorted([data.num_nodes for data in train_dataset])
#             k = num_nodes[int(math.ceil(k * len(num_nodes))) - 1]
#             k = int(max(10, k))

#         self.convs = ModuleList()
#         self.convs.append(GNN(train_dataset.num_features, hidden_dim))
#         for i in range(0, num_layers - 1):
#             self.convs.append(GNN(hidden_dim, hidden_dim))
#         self.convs.append(GNN(hidden_dim, 1))

#         conv1d_channels = [16, 32]
#         total_latent_dim = hidden_dim * num_layers + 1
#         conv1d_kws = [total_latent_dim, 5]
#         self.conv1 = Conv1d(1, conv1d_channels[0], conv1d_kws[0],
#                             conv1d_kws[0])
#         self.pool = SortAggregation(k)
#         self.maxpool1d = MaxPool1d(2, 2)
#         self.conv2 = Conv1d(conv1d_channels[0], conv1d_channels[1],
#                             conv1d_kws[1], 1)
#         dense_dim = int((k - 2) / 2 + 1)
#         dense_dim = (dense_dim - conv1d_kws[1] + 1) * conv1d_channels[1]
#         self.mlp = MLP([dense_dim, 128, 1], dropout=0.5, norm=None)

#     def forward(self, x, edge_index, batch):
#         xs = [x]
#         for conv in self.convs:
#             xs += [conv(xs[-1], edge_index).tanh()]
#         x = torch.cat(xs[1:], dim=-1)

#         # Global pooling.
#         x = self.pool(x, batch)
#         x = x.unsqueeze(1)  # [num_graphs, 1, k * hidden]
#         x = self.conv1(x).relu()
#         x = self.maxpool1d(x)
#         x = self.conv2(x).relu()
#         x = x.view(x.size(0), -1)  # [num_graphs, dense_dim]

#         return self.mlp(x)


In [21]:
scorer = NeighborScorer(input_dim=data.x.size(1), hidden_dim=64).to(device)
model = DGCNN(hidden_dim=32, num_layers=3, dataset=all_edges).to(device)
optimizer = torch.optim.Adam(list(model.parameters()) + list(scorer.parameters()), lr=0.001)
criterion = nn.BCEWithLogitsLoss()

AttributeError: 'tuple' object has no attribute 'num_nodes'

In [ ]:
batch_size = 32
num_epochs = 10
num_hops = 2
top_n = 10

for epoch in range(num_epochs):
    model.train()
    scorer.train()
    total_loss = 0

    for i in range(0, len(all_edges), batch_size):
        batch_edges = all_edges[i:i+batch_size]
        batch_data_list = []
        for src, dst, label in batch_edges:
            x_sub, edge_index_sub, z_sub = dynamic_prune_subgraph(
                src, dst, data, num_hops, scorer, top_n
            )
            batch_data_list.append(
                Data(x=x_sub, z=z_sub, edge_index=edge_index_sub,
                     y=torch.tensor([label], dtype=torch.float))
            )
        batch = DataLoader(batch_data_list, batch_size=len(batch_data_list), shuffle=False)
        for data_sub in batch:
            data_sub = data_sub.to(device)
            optimizer.zero_grad()
            out = model(data_sub.x, data_sub.edge_index, data_sub.batch)
            loss = criterion(out.view(-1), data_sub.y)
            loss.backward()
            optimizer.step()
            total_loss += float(loss) * data_sub.num_graphs

    print(f'Epoch {epoch}, Loss {total_loss / len(all_edges):.4f}')


In [ ]:
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# model = DGCNN(hidden_dim=32, num_layers=3).to(device)
# optimizer = torch.optim.Adam(list(model.parameters()) + list(scorer.parameters()), lr=0.0001)
# criterion = BCEWithLogitsLoss()

In [ ]:
# def train():
#     model.train()
#     scorer.train()
#     total_loss = 0
    
#     for data in train_loader:
#         data = data.to(device)
#         optimizer.zero_grad()
#         out = model(data.x, data.edge_index, data.batch)
#         loss = criterion(out.view(-1), data.y.to(torch.float))
#         loss.backward()
#         optimizer.step()
#         total_loss += float(loss) * data.num_graphs

#     return total_loss / len(train_dataset)

In [ ]:
@torch.no_grad()
def test(loader):
    model.eval()

    y_pred, y_true = [], []
    for data in loader:
        data = data.to(device)
        logits = model(data.x, data.edge_index, data.batch)
        y_pred.append(logits.view(-1).cpu())
        y_true.append(data.y.view(-1).cpu().to(torch.float))

    return roc_auc_score(torch.cat(y_true), torch.cat(y_pred)), average_precision_score(torch.cat(y_true),torch.cat(y_pred) )

In [ ]:
# times = []
# best_val_auc = test_auc = 0
# for epoch in range(1, 51):
#     start = time.time()
#     loss = train()
#     val_auc, val_ap = test(val_loader)
#     if val_auc > best_val_auc:
#         best_val_auc = val_auc
#         test_auc, test_ap = test(test_loader)
#     print(f'Epoch: {epoch:02d}, Loss: {loss:.4f}, Val_AUC: {val_auc:.4f}, Val_AP: {val_ap:.4f}'
#           f'Test_AUC: {test_auc:.4f}, Test_AP: {test_ap:.4f}')
#     times.append(time.time() - start)
# print(f"Median time per epoch: {torch.tensor(times).median():.4f}s")